In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

In [3]:
spark=SparkSession.builder \
.master('local[4]') \
.appName('Dataset oluşturmak') \
.config('spark.executer.memory',"4g") \
.config('spark.driver.memory',"2g") \
.getOrCreate()

sc= spark.sparkContext

In [4]:
retail_rdd=sc.textFile('C:/Users/Goksun/Downloads/OnlineRetail.csv')

In [5]:
retail_rdd2=retail_rdd.filter(lambda x:"InvoiceNo" not in x)

In [6]:
retail_rdd2.take(5)

['536365;85123A;WHITE HANGING HEART T-LIGHT HOLDER;6;1.12.2010 08:26;2,55;17850;United Kingdom',
 '536365;71053;WHITE METAL LANTERN;6;1.12.2010 08:26;3,39;17850;United Kingdom',
 '536365;84406B;CREAM CUPID HEARTS COAT HANGER;8;1.12.2010 08:26;2,75;17850;United Kingdom',
 '536365;84029G;KNITTED UNION FLAG HOT WATER BOTTLE;6;1.12.2010 08:26;3,39;17850;United Kingdom',
 '536365;84029E;RED WOOLLY HOTTIE WHITE HEART.;6;1.12.2010 08:26;3,39;17850;United Kingdom']

In [8]:
def cancel_price(line):
    is_cancelled= True if (line.split(";")[0].startswith('C')) else False
    quantity= float(line.split(";")[3])
    price= float(line.split(";")[5].replace(",","."))
    total=quantity*price
    return (is_cancelled, total)

In [9]:
cancelled_total= retail_rdd2.map(cancel_price)

In [10]:
cancelled_total.take(5)

[(False, 15.299999999999999),
 (False, 20.34),
 (False, 22.0),
 (False, 20.34),
 (False, 20.34)]

In [11]:
cancelled_total_reduce=cancelled_total.reduceByKey(lambda x,y: x+y)

In [12]:
cancelled_total_reduce.take(3)

[(False, 10644560.424000263), (True, -896812.4900000116)]

In [13]:
cancelled_total_reduce.filter(lambda x: x[0]==True).take(2)

[(True, -896812.4900000116)]

In [14]:
cancelled_total_reduce.filter(lambda x: x[0]==True).map(lambda x: x[1]).take(2)

[-896812.4900000116]